In [204]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
import pandas_profiling as pp
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import xgboost
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [205]:
pd.set_option('display.max_columns', 1000)

In [206]:
def onehotencode(data,col_name,prefix):
    return pd.concat([data, pd.get_dummies(data[col_name], prefix=prefix)],axis=1)

In [207]:
def encode_total_stops(data):
    data['Encoded_stops'] = 0
    for i in range(0,len(data['Total_Stops'])):
        if data.loc[i, 'Total_Stops'] == '1 stop':
            data.loc[i,'Encoded_stops'] = 1
        elif data.loc[i, 'Total_Stops'] == '2 stops':
            data.loc[i,'Encoded_stops'] = 2
        elif data.loc[i, 'Total_Stops'] == '3 stops' or data.loc[i, 'Total_Stops'] == '':
            data.loc[i,'Encoded_stops'] = 3
        elif data.loc[i, 'Total_Stops'] == '4 stops':
            data.loc[i,'Encoded_stops'] = 4
        elif data.loc[i, 'Total_Stops'] == 'non-stop':
            data.loc[i,'Encoded_stops'] = 0

    return data

In [208]:
def extract_journey_data(data):
    data['Journey_Day'] = pd.DataFrame(pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.day)
    data['Journey_Month'] = pd.DataFrame(pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.month)
    data['Journey_day_of_week'] =  pd.DataFrame(((pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.weekday_name)))
#   data['Journey_over_weekend'] =  pd.DataFrame(((pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.dayofweek) // 5).astype('int'))
    data['Date_Lapse'] = ((pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y')) - 
                                        (pd.to_datetime('01/03/2019', format='%d/%m/%Y')))/np.timedelta64(1,'D')
        
    return data

In [209]:
def encode_duration(data):
    data['Duration_Type'] = 0
    for i in range(0, len(data)):
    
        split_len = data.loc[i,'Duration'].split(' ')
        if len(split_len) == 1:
            if str(data.loc[i,'Duration']).find('h') != -1:
#                print ("hours: ",data.loc[i,'Duration'])
                hours = int(split_len[0].replace('h',""))
                mins = 0
            else:
                hours = 4 
#                print ("mins: ",data.loc[i,'Duration']
                mins = int(split_len[0].replace('m',""))
        else:
            
            hours = int(split_len[0].replace('h',""))
            mins  = int(split_len[1].replace('m',""))
        
        data.loc[i,'Duration_Type'] = (hours*60 + mins)/1440
        
    return data

In [210]:
'''
def encode_duration(data):
    data_copy['Duration_Type'] = 0
    for i in range(0, len(data)):
    
        try:
            val = int(str(data.loc[i,'Duration'].split('h')[0]))
        except:
            # this is for AirIndia for with flight duration as '5m', changing the duration
            #based on same row for AirIndia for same destination at same time with same stop overs.
            val = 4
            
       
        if val < 3:
            data.loc[i,'Duration_Type'] = 0.0
        elif val < 6:
            data.loc[i,'Duration_Type'] = 0.1
        elif val < 12:
            data.loc[i,'Duration_Type'] = 0.2
        else:
            data.loc[i,'Duration_Type'] = 0.3
        
    return data
'''

"\ndef encode_duration(data):\n    data_copy['Duration_Type'] = 0\n    for i in range(0, len(data)):\n    \n        try:\n            val = int(str(data.loc[i,'Duration'].split('h')[0]))\n        except:\n            # this is for AirIndia for with flight duration as '5m', changing the duration\n            #based on same row for AirIndia for same destination at same time with same stop overs.\n            val = 4\n            \n       \n        if val < 3:\n            data.loc[i,'Duration_Type'] = 0.0\n        elif val < 6:\n            data.loc[i,'Duration_Type'] = 0.1\n        elif val < 12:\n            data.loc[i,'Duration_Type'] = 0.2\n        else:\n            data.loc[i,'Duration_Type'] = 0.3\n        \n    return data\n"

In [211]:
'''
def flight_hours(data):
    data['Fly_Hours'] = 0
    
    data_Dep_Time = data['Dep_Time'].str.split(":",n=1,expand=True).astype('int64')
    data_Dep_Time.columns = ['Hours','Mins']
    for i in range(0,len(data)):
        
        val = data_Dep_Time.loc[i, 'Hours']
        
        if val == 0:
            data.loc[i,'Fly_Hours'] = 'Midnight'
        elif val >= 1 and val < 4:
            data.loc[i,'Fly_Hours'] = 'Early_Morning'
        elif val >= 4 and val < 6:
            data.loc[i,'Fly_Hours'] = 'Dawn'
        elif val >= 6 and val < 9:
            data.loc[i,'Fly_Hours'] = 'Morning'
        elif val >= 9 and val < 12:
            data.loc[i,'Fly_Hours'] = 'Mid_Morning'
        elif val >= 12 and val < 13:
            data.loc[i,'Fly_Hours'] = 'Noon'
        elif val >= 13 and val < 16:
            data.loc[i,'Fly_Hours'] = 'After_Noon'
        elif val >= 16 and val < 21:
            data.loc[i,'Fly_Hours'] = 'Evening'
        elif val >= 21 and val <= 23:
            data.loc[i,'Fly_Hours'] = 'Night'
            
    return data
'''

'\ndef flight_hours(data):\n    data[\'Fly_Hours\'] = 0\n    \n    data_Dep_Time = data[\'Dep_Time\'].str.split(":",n=1,expand=True).astype(\'int64\')\n    data_Dep_Time.columns = [\'Hours\',\'Mins\']\n    for i in range(0,len(data)):\n        \n        val = data_Dep_Time.loc[i, \'Hours\']\n        \n        if val == 0:\n            data.loc[i,\'Fly_Hours\'] = \'Midnight\'\n        elif val >= 1 and val < 4:\n            data.loc[i,\'Fly_Hours\'] = \'Early_Morning\'\n        elif val >= 4 and val < 6:\n            data.loc[i,\'Fly_Hours\'] = \'Dawn\'\n        elif val >= 6 and val < 9:\n            data.loc[i,\'Fly_Hours\'] = \'Morning\'\n        elif val >= 9 and val < 12:\n            data.loc[i,\'Fly_Hours\'] = \'Mid_Morning\'\n        elif val >= 12 and val < 13:\n            data.loc[i,\'Fly_Hours\'] = \'Noon\'\n        elif val >= 13 and val < 16:\n            data.loc[i,\'Fly_Hours\'] = \'After_Noon\'\n        elif val >= 16 and val < 21:\n            data.loc[i,\'Fly_Hours\']

In [212]:
def flight_hours(data):
    data['Fly_Hours'] = 0
    
    data_Dep_Time = data['Dep_Time'].str.split(":",n=1,expand=True).astype('int64')
    data_Dep_Time.columns = ['Hours','Mins']
    for i in range(0,len(data)):
        
        hr = data_Dep_Time.loc[i, 'Hours']
        min = data_Dep_Time.loc[i, 'Mins']
        data.loc[i,'Fly_Hours'] = (hr*60 + (min))/1440
            
    return data

In [213]:
def arrival_hours(data):
    data['Arrival_Hours'] = 0
    for i in range(0,len(data)):
      
        data_Arr_Time = data.loc[i,'Arrival_Time'].split(" ")[0]
        data_Dep_Time = data_Arr_Time.split(":")
        
        hr = int(data_Dep_Time[0])
        min = int(data_Dep_Time[1])
        data.loc[i,'Arrival_Hours'] = (hr*60 + (min))/1440
            
    return data

In [214]:
def log_transform(data):
    return np.log10(data)

# Data cleaning and Modeling starts here

In [215]:
data = pd.read_excel('C:\\Users\\LENOVO\\Desktop\\Flight_Ticket_Participant_Datasets\\Data_Train.xlsx')

In [216]:
test = pd.read_excel('C:\\Users\\LENOVO\\Desktop\\Flight_Ticket_Participant_Datasets\\Test_set.xlsx')

In [217]:
data_copy = data
test_copy = test

In [218]:
data_copy.shape

(10683, 11)

In [219]:
data_copy.drop_duplicates(keep='first', inplace=True)

In [220]:
data_copy.shape

(10463, 11)

In [221]:
#Drop anamolies.
#data_copy = data_copy[data_copy['Price'] > data["Price"].quantile(0.01)]
#data_copy = data_copy[data_copy['Price'] < data["Price"].quantile(0.99)]
data_copy = data_copy[data_copy['Airline'] != 'Trujet']
data_copy = data_copy[data_copy['Additional_Info'] != 'Red-eye flight']

In [222]:
#data_copy.loc[data_copy['Airline'] == 'Jet Airways Business','Airline'] = 'Jet Airways'
#data_copy.loc[data_copy['Airline'] == 'Multiple carriers Premium economy','Airline'] = 'Multiple carriers'
#data_copy.loc[data_copy['Airline'] == 'Vistara Premium economy','Airline'] = 'Vistara'

In [223]:
#data_copy.loc[data_copy['Destination'] == 'New Delhi','Destination'] = 'Delhi'

In [224]:
data_copy.reset_index(inplace=True)

In [225]:
data_copy = data_copy[data_copy['Airline'] != 'Trujet']

In [226]:
data_copy = onehotencode(data_copy,'Airline',"Airline_")

In [227]:
data_copy = onehotencode(data_copy,'Source',"Source_")

In [228]:
data_copy = onehotencode(data_copy,'Destination',"Destination_")

In [229]:
#handle No Info, No info values before one hot encoding.
data_copy.loc[data_copy['Additional_Info'] == 'No info','Additional_Info'] = 'No Info'
data_copy.loc[data_copy['Additional_Info'] == '1 Long layover','Additional_Info'] = 'Layover'
data_copy.loc[data_copy['Additional_Info'] == '1 Short layover','Additional_Info'] = 'Layover'
data_copy.loc[data_copy['Additional_Info'] == '2 Long layover','Additional_Info'] = 'Layover'
data_copy.loc[data_copy['Additional_Info'] == 'Business class','Additional_Info'] = 'Business_Class'
data_copy.loc[data_copy['Additional_Info'] == 'Change airports','Additional_Info'] = 'Airport_Changed'


data_copy = onehotencode(data_copy,'Additional_Info',"Additional_Info_")

In [230]:
data_copy = encode_total_stops(data_copy)

In [231]:
data_copy = extract_journey_data(data_copy)

In [232]:
data_copy = onehotencode(data_copy,'Journey_day_of_week',"Dayis_")

In [233]:
data_copy['Month_Start'] = 0
data_copy.loc[data_copy['Journey_Day'] == 1,'Month_Start'] = 1
data_copy.loc[data_copy['Journey_Day'] == 3,'Month_Start'] = 1
data_copy.loc[data_copy['Journey_Day'] == 6,'Month_Start'] = 1
data_copy.loc[data_copy['Journey_Day'] == 9,'Month_Start'] = 1

In [234]:
#data_copy['is_event'] = 0
#data_copy['is_event'] = np.where((((data_copy['Journey_Month'] == 3) & (data_copy['Journey_Day'] == 24)) |
#                                 ((data_copy['Journey_Month'] == 4) & (data_copy['Journey_Day'] == 21)) |
#                                 ((data_copy['Journey_Month'] == 5) & (data_copy['Journey_Day'] == 18))),1,0)

In [235]:
#data_copy['is_event'].value_counts()

In [236]:
data_copy = encode_duration(data_copy)

In [237]:
data_copy = flight_hours(data_copy)

In [238]:
data_copy = arrival_hours(data_copy)

In [239]:
data_copy = onehotencode(data_copy,'Encoded_stops',"No_of_stops_")

In [240]:
#data_copy = onehotencode(data_copy,'Fly_Hours',"Fly_Hours_")

In [241]:
data_final = data_copy

In [242]:
#data_final.loc[data_final['Encoded_stops'].isnull(),'Encoded_stops'] = 0.3

In [243]:
data_final.drop(['Airline','Date_of_Journey','Source','Destination','Route','Dep_Time','Arrival_Time','Duration',
                'Total_Stops','Additional_Info','Journey_day_of_week','Encoded_stops'],axis=1,inplace=True)
#                'Total_Stops','Additional_Info','Fly_Hours'],axis=1,inplace=True)

In [244]:
data_final.drop(['index'],axis=1,inplace=True)

In [245]:
train_Feature = data_copy[[x for x in data_final.columns if (x != 'Price' and x != 'index')]]
train_Target = data_copy['Price']

In [246]:
X_train, X_test, y_train, y_test = train_test_split(train_Feature, train_Target, test_size=0.30, random_state=101)

In [247]:
null_columns=X_train.columns[X_train.isnull().any()]
X_train[null_columns].isnull().sum()

Series([], dtype: float64)

In [248]:
X_train.head()

,Airline__Air Asia,Airline__Air India,Airline__GoAir,Airline__IndiGo,Airline__Jet Airways,Airline__Jet Airways Business,Airline__Multiple carriers,Airline__Multiple carriers Premium economy,Airline__SpiceJet,Airline__Vistara,Airline__Vistara Premium economy,Source__Banglore,Source__Chennai,Source__Delhi,Source__Kolkata,Source__Mumbai,Destination__Banglore,Destination__Cochin,Destination__Delhi,Destination__Hyderabad,Destination__Kolkata,Destination__New Delhi,Additional_Info__Airport_Changed,Additional_Info__Business_Class,Additional_Info__In-flight meal not included,Additional_Info__Layover,Additional_Info__No Info,Additional_Info__No check-in baggage included,Journey_Day,Journey_Month,Date_Lapse,Dayis__Friday,Dayis__Monday,Dayis__Saturday,Dayis__Sunday,Dayis__Thursday,Dayis__Tuesday,Dayis__Wednesday,Month_Start,Duration_Type,Fly_Hours,Arrival_Hours,No_of_stops__0,No_of_stops__1,No_of_stops__2,No_of_stops__3,No_of_stops__4
6949,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,21,6,112.0,1,0,0,0,0,0,0,0,0.062500,0.579861,0.642361,1,0,0,0,0
8604,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,9,6,100.0,0,0,0,1,0,0,0,1,0.118056,0.996528,0.114583,1,0,0,0,0
883,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,9,5,69.0,0,0,0,0,1,0,0,1,0.479167,0.312500,0.791667,0,1,0,0,0
4116,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,3,0.0,1,0,0,0,0,0,0,1,0.638889,0.916667,0.555556,0,1,0,0,0
5606,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,27,6,118.0,0,0,0,0,1,0,0,0,0.375000,0.416667,0.791667,0,1,0,0,0


In [249]:
scaler = StandardScaler()

In [250]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=train_Feature.columns)

In [251]:
X_test =  pd.DataFrame(scaler.transform(X_test),columns=train_Feature.columns)

# Final Prediction on Test set starts here

In [252]:
#test_copy.loc[test_copy['Airline'] == 'Jet Airways Business','Airline'] = 'Jet Airways'
#test_copy.loc[test_copy['Airline'] == 'Multiple carriers Premium economy','Airline'] = 'Multiple carriers'
#test_copy.loc[test_copy['Airline'] == 'Vistara Premium economy','Airline'] = 'Vistara'

In [253]:
#test_copy.loc[test_copy['Destination'] == 'New Delhi','Destination'] = 'Delhi'

In [254]:
test_copy = onehotencode(test_copy,'Airline',"Airline_")

In [255]:
test_copy = onehotencode(test_copy,'Source',"Source_")

In [256]:
test_copy = onehotencode(test_copy,'Destination',"Destination_")

In [257]:
#handle No Info, No info values before one hot encoding.
test_copy.loc[test_copy['Additional_Info'] == 'No info','Additional_Info'] = 'No Info'
test_copy.loc[test_copy['Additional_Info'] == '1 Long layover','Additional_Info'] = 'Layover'
test_copy.loc[test_copy['Additional_Info'] == '1 Short layover','Additional_Info'] = 'Layover'
test_copy.loc[test_copy['Additional_Info'] == '1 Long layover','Additional_Info'] = 'Layover'
test_copy.loc[test_copy['Additional_Info'] == 'Business class','Additional_Info'] = 'Business_Class'
test_copy.loc[test_copy['Additional_Info'] == 'Change airports','Additional_Info'] = 'Airport_Changed'

test_copy = onehotencode(test_copy,'Additional_Info',"Additional_Info_")

In [258]:
test_copy = encode_total_stops(test_copy)

In [259]:
test_copy = extract_journey_data(test_copy)

In [260]:
test_copy['Month_Start'] = 0
test_copy.loc[test_copy['Journey_Day'] == 1,'Month_Start'] = 1
test_copy.loc[test_copy['Journey_Day'] == 3,'Month_Start'] = 1
test_copy.loc[test_copy['Journey_Day'] == 6,'Month_Start'] = 1
test_copy.loc[test_copy['Journey_Day'] == 9,'Month_Start'] = 1

In [261]:
#test_copy['is_event'] = 0
#test_copy['is_event'] = np.where((((test_copy['Journey_Month'] == 3) & (test_copy['Journey_Day'] == 24)) |
#                                 ((test_copy['Journey_Month'] == 4) & (test_copy['Journey_Day'] == 21)) |
#                                 ((test_copy['Journey_Month'] == 5) & (test_copy['Journey_Day'] == 18))),1,0)

In [262]:
test_copy = encode_duration(test_copy)

In [263]:
test_copy = flight_hours(test_copy)

In [264]:
test_copy = arrival_hours(test_copy)

In [265]:
test_copy = onehotencode(test_copy,'Journey_day_of_week',"Dayis_")

In [266]:
test_copy = onehotencode(test_copy,'Encoded_stops',"No_of_stops_")

In [267]:
#test_copy = onehotencode(test_copy,'Fly_Hours',"Fly_Hours_")

In [268]:
test_final = test_copy

In [269]:
data_copy.shape

(10461, 48)

In [270]:
[X for X in data_copy.columns if X not in test_final.columns]

['Price']

In [271]:
data_copy.head()

,Price,Airline__Air Asia,Airline__Air India,Airline__GoAir,Airline__IndiGo,Airline__Jet Airways,Airline__Jet Airways Business,Airline__Multiple carriers,Airline__Multiple carriers Premium economy,Airline__SpiceJet,Airline__Vistara,Airline__Vistara Premium economy,Source__Banglore,Source__Chennai,Source__Delhi,Source__Kolkata,Source__Mumbai,Destination__Banglore,Destination__Cochin,Destination__Delhi,Destination__Hyderabad,Destination__Kolkata,Destination__New Delhi,Additional_Info__Airport_Changed,Additional_Info__Business_Class,Additional_Info__In-flight meal not included,Additional_Info__Layover,Additional_Info__No Info,Additional_Info__No check-in baggage included,Journey_Day,Journey_Month,Date_Lapse,Dayis__Friday,Dayis__Monday,Dayis__Saturday,Dayis__Sunday,Dayis__Thursday,Dayis__Tuesday,Dayis__Wednesday,Month_Start,Duration_Type,Fly_Hours,Arrival_Hours,No_of_stops__0,No_of_stops__1,No_of_stops__2,No_of_stops__3,No_of_stops__4
0,3897,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,24,3,23.0,0,0,0,1,0,0,0,0,0.118056,0.930556,0.048611,1,0,0,0,0
1,7662,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,5,61.0,0,0,0,0,0,0,1,1,0.309028,0.243056,0.552083,0,0,1,0,0
2,13882,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,9,6,100.0,0,0,0,1,0,0,0,1,0.791667,0.392361,0.184028,0,0,1,0,0
3,6218,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,12,5,72.0,0,0,0,1,0,0,0,0,0.225694,0.753472,0.979167,0,1,0,0,0
4,13302,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,3,0.0,1,0,0,0,0,0,0,1,0.197917,0.701389,0.899306,0,1,0,0,0


In [272]:
test_final.drop(['Airline','Date_of_Journey','Source','Destination','Route','Dep_Time','Arrival_Time','Duration',
                'Total_Stops','Additional_Info','Journey_day_of_week','Encoded_stops'],axis=1,inplace=True)
#                'Total_Stops','Additional_Info','Fly_Hours'],axis=1,inplace=True)

In [273]:
test_final =  pd.DataFrame(scaler.transform(test_final),columns=test_final.columns)

In [274]:
test_final.columns

Index(['Airline__Air Asia', 'Airline__Air India', 'Airline__GoAir',
       'Airline__IndiGo', 'Airline__Jet Airways',
       'Airline__Jet Airways Business', 'Airline__Multiple carriers',
       'Airline__Multiple carriers Premium economy', 'Airline__SpiceJet',
       'Airline__Vistara', 'Airline__Vistara Premium economy',
       'Source__Banglore', 'Source__Chennai', 'Source__Delhi',
       'Source__Kolkata', 'Source__Mumbai', 'Destination__Banglore',
       'Destination__Cochin', 'Destination__Delhi', 'Destination__Hyderabad',
       'Destination__Kolkata', 'Destination__New Delhi',
       'Additional_Info__Airport_Changed', 'Additional_Info__Business_Class',
       'Additional_Info__In-flight meal not included',
       'Additional_Info__Layover', 'Additional_Info__No Info',
       'Additional_Info__No check-in baggage included', 'Journey_Day',
       'Journey_Month', 'Date_Lapse', 'Month_Start', 'Duration_Type',
       'Fly_Hours', 'Arrival_Hours', 'Dayis__Friday', 'Dayis__Monday',


In [275]:
train_Feature.columns

Index(['Airline__Air Asia', 'Airline__Air India', 'Airline__GoAir',
       'Airline__IndiGo', 'Airline__Jet Airways',
       'Airline__Jet Airways Business', 'Airline__Multiple carriers',
       'Airline__Multiple carriers Premium economy', 'Airline__SpiceJet',
       'Airline__Vistara', 'Airline__Vistara Premium economy',
       'Source__Banglore', 'Source__Chennai', 'Source__Delhi',
       'Source__Kolkata', 'Source__Mumbai', 'Destination__Banglore',
       'Destination__Cochin', 'Destination__Delhi', 'Destination__Hyderabad',
       'Destination__Kolkata', 'Destination__New Delhi',
       'Additional_Info__Airport_Changed', 'Additional_Info__Business_Class',
       'Additional_Info__In-flight meal not included',
       'Additional_Info__Layover', 'Additional_Info__No Info',
       'Additional_Info__No check-in baggage included', 'Journey_Day',
       'Journey_Month', 'Date_Lapse', 'Dayis__Friday', 'Dayis__Monday',
       'Dayis__Saturday', 'Dayis__Sunday', 'Dayis__Thursday', 'Dayis_

In [276]:
test_final.shape

(2671, 47)

In [277]:
train_Feature.shape

(10461, 47)

In [278]:
y_train_log =  y_train.apply(log_transform)

In [279]:
y_test_log =  y_test.apply(log_transform)

## RandomSearch

In [280]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [281]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [282]:
rf = RandomForestRegressor()

In [283]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [284]:
rf_random.fit(X_train, y_train_log)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 43.7min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'min_samples_leaf': [1, 2, 4], 'max_depth': [10, 20, 30, 40, 50, None], 'n_estimators': [200, 400, 600, 800, 1000], 'bootstrap': [True, False], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [285]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 800}

In [286]:
np.sqrt(mean_squared_log_error(y_test,((10**rf_random.predict(X_test)))))

0.12686510836665554

In [287]:
np.sqrt(mean_squared_log_error(y_train,10**rf_random.predict(X_train)))

0.05320737206746657

In [288]:
test_pred = 10**rf_random.predict(test_final)

In [289]:
test_pred = pd.DataFrame(test_pred,columns=['Price'])

In [290]:
test_pred.to_excel("C:/Users/LENOVO/Desktop/RF_Random_v2.xlsx",index=False)

In [291]:
test_copy.head()

,Airline__Air Asia,Airline__Air India,Airline__GoAir,Airline__IndiGo,Airline__Jet Airways,Airline__Jet Airways Business,Airline__Multiple carriers,Airline__Multiple carriers Premium economy,Airline__SpiceJet,Airline__Vistara,Airline__Vistara Premium economy,Source__Banglore,Source__Chennai,Source__Delhi,Source__Kolkata,Source__Mumbai,Destination__Banglore,Destination__Cochin,Destination__Delhi,Destination__Hyderabad,Destination__Kolkata,Destination__New Delhi,Additional_Info__Airport_Changed,Additional_Info__Business_Class,Additional_Info__In-flight meal not included,Additional_Info__Layover,Additional_Info__No Info,Additional_Info__No check-in baggage included,Journey_Day,Journey_Month,Date_Lapse,Month_Start,Duration_Type,Fly_Hours,Arrival_Hours,Dayis__Friday,Dayis__Monday,Dayis__Saturday,Dayis__Sunday,Dayis__Thursday,Dayis__Tuesday,Dayis__Wednesday,No_of_stops__0,No_of_stops__1,No_of_stops__2,No_of_stops__3,No_of_stops__4
0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,6,6,97.0,1,0.454861,0.729167,0.184028,0,0,0,0,1,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,12,5,72.0,0,0.166667,0.263889,0.430556,0,0,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,21,5,81.0,0,0.989583,0.802083,0.791667,0,0,0,0,0,1,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,21,5,81.0,0,0.541667,0.333333,0.875000,0,0,0,0,0,1,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,24,6,115.0,0,0.118056,0.996528,0.114583,0,1,0,0,0,0,0,1,0,0,0,0


## Grid Search

In [292]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [60,70,80,90,100],
    'max_features': [2,3],
    'min_samples_leaf': [5,6],
    'min_samples_split': [10,12],
    'n_estimators': [600,800,1000,1200,1400]
}

In [293]:
rf = RandomForestRegressor()

In [ ]:
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X_train, y_train_log)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 23.5min


In [ ]:
grid_search.best_params_

In [ ]:
y_pred_test = grid_search.predict(X_test)

In [ ]:
y_pred_train = grid_search.predict(X_train)

In [ ]:
np.sqrt(mean_squared_log_error(y_test,10**grid_search.predict(X_test)))

In [ ]:
np.sqrt(mean_squared_log_error(y_train,10**grid_search.predict(X_train)))

In [ ]:
test_pred = 10**grid_search.predict(test_final)

In [ ]:
test_pred = pd.DataFrame(test_pred,columns=['Price'])

In [ ]:
test_pred.to_excel("C:/Users/LENOVO/Desktop/RF_Grid_v1.xlsx",index=False)

In [ ]:
for i in zip(X_train.columns,grid_search.best_estimator_.feature_importances_):
    print (i)